In [4]:
library(kernlab)
library(finalfit)
# library(DataExplorer)
# library(doBy)
library(dplyr)
library(tidyr)
library(caret)
library(MASS)
# library(klaR)
# library(ggord)
library(corrplot)

Warning message:
"pakiet 'corrplot' został zbudowany w wersji R 4.3.2"
corrplot 0.92 loaded



In [5]:
data(spam)

In [6]:
train_indices <- as.matrix(read.csv("ti.csv"))

In [7]:
numeric_columns <- sapply(spam, is.numeric)
spam_numeric <- spam[, numeric_columns]
scaled_data <- as.data.frame(scale(spam_numeric))
spam_scaled <- cbind(scaled_data, spam[, !numeric_columns, drop = FALSE])

## Commands to put to knitr

#### columns elimination

In [8]:
correlatedFeatures.id <- findCorrelation(cor(spam[, 1:57]), names=TRUE, cutoff=0.8, verbose=TRUE)

Compare row 32  and column  34 with corr  0.996 
  Means:  0.143 vs 0.059 so flagging column 32 
Compare row 34  and column  40 with corr  0.845 
  Means:  0.127 vs 0.057 so flagging column 34 
All correlations <= 0.8 


In [9]:
correlatedFeatures.id

[1] "num857" "num415"

In [10]:
spam_scaled <- spam_scaled[, -c(32, 34)]

#### correlation with spam 

In [11]:
spam_binary <- spam[, -56]
spam_binary$spam <- ifelse(spam$type == "spam", 1, 0)

In [12]:
correlation <- cor(spam_binary)
correlation_with_spam <- correlation[, ncol(correlation)]

ERROR: Error in cor(spam_binary): argument 'x' musi być liczbą


In [ ]:
correlation_with_spam <- abs(correlation_with_spam)
correlation_with_spam <- sort(correlation_with_spam, decreasing = TRUE)
most_correlated <- data.frame(correlation_with_spam[2:11])

In [13]:
most_correlated

ERROR: Error in eval(expr, envir, enclos): nie znaleziono obiektu 'most_correlated'


#### average value for type

In [15]:
spam_data <- spam[spam$type == "spam", ]
nonspam_data <- spam[spam$type == "nonspam", ]

freq_spam <- colMeans(spam_data[, 1:52])
freq_nonspam <- colMeans(nonspam_data[, 1:52])

freq_diff <- abs(freq_spam - freq_nonspam)
sorted_freqs <- sort(freq_diff, decreasing = TRUE)[1:10]

freq_spam_df <- t(as.data.frame(freq_spam))
freq_nonspam_df <- t(as.data.frame(freq_nonspam))

colnames <- names(sorted_freqs)
freq_spam_highest <- freq_spam_df[, c(colnames[1])]

freqs_df <- data.frame(c('spam', 'nonspam'))
colnames(freqs_df) <- c('type')
for (i in 1:length(sorted_freqs)) {
  column <- c(freq_spam_df[, c(colnames[i])], freq_nonspam_df[, c(colnames[i])])

  freqs_df[colnames[i]] <- column
}

In [16]:
freqs_df

type,george,you,your,hp,free,hpl,charExclamation,our,re,edu
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
spam,0.001549917,2.264539,1.3803696,0.01747932,0.5183618,0.009172642,0.5137126,0.5139548,0.1250910,0.01472697
nonspam,1.265265423,1.270341,0.4387016,0.89547346,0.0735868,0.431994261,0.1099835,0.1810402,0.4157604,0.28718436


## Discriminant analysis

#### whole data set

In [140]:
train <- spam_scaled[train_indices, ]
test  <- spam_scaled[-train_indices, ]

In [141]:
train <- data.frame(train)
test <- data.frame(test)
lda_fit <- lda(type ~ ., data = train)
lda_predictions <- predict(lda_fit, test)

In [142]:
qda_fit <- qda(type ~ ., data = train)
qda_predictions <- predict(qda_fit, test)

In [143]:
lda_cm <- confusionMatrix(test$type, lda_predictions$class, positive = "spam")
qda_cm <- confusionMatrix(test$type, qda_predictions$class, positive = "spam")

In [144]:
lda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     661   36
   spam         90  363
                                          
               Accuracy : 0.8904          
                 95% CI : (0.8709, 0.9079)
    No Information Rate : 0.653           
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7657          
                                          
 Mcnemar's Test P-Value : 2.34e-06        
                                          
            Sensitivity : 0.9098          
            Specificity : 0.8802          
         Pos Pred Value : 0.8013          
         Neg Pred Value : 0.9484          
             Prevalence : 0.3470          
         Detection Rate : 0.3157          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.8950          
                                          
       'Positive' Class : spam            
               

In [145]:
qda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     539  158
   spam         18  435
                                          
               Accuracy : 0.847           
                 95% CI : (0.8248, 0.8673)
    No Information Rate : 0.5157          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.6959          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.7336          
            Specificity : 0.9677          
         Pos Pred Value : 0.9603          
         Neg Pred Value : 0.7733          
             Prevalence : 0.5157          
         Detection Rate : 0.3783          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.8506          
                                          
       'Positive' Class : spam            
               

## Whole data set with log

In [169]:
numeric_columns <- spam[, 1:55]
log_data <- as.data.frame(log(numeric_columns+0.1))
spam_log <- cbind(log_data, spam[, 56, drop = FALSE])

In [170]:
train <- spam_log[train_indices, ]
test  <- spam_log[-train_indices, ]

In [171]:
train <- data.frame(train)
test <- data.frame(test)
lda_fit <- lda(type ~ ., data = train)
lda_predictions <- predict(lda_fit, test)
qda_fit <- qda(type ~ ., data = train)
qda_predictions <- predict(qda_fit, test)

In [172]:
lda_cm <- confusionMatrix(test$type, lda_predictions$class, positive = "spam")
qda_cm <- confusionMatrix(test$type, qda_predictions$class, positive = "spam")

In [173]:
lda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     675   22
   spam         53  400
                                          
               Accuracy : 0.9348          
                 95% CI : (0.9189, 0.9484)
    No Information Rate : 0.633           
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.8618          
                                          
 Mcnemar's Test P-Value : 0.000532        
                                          
            Sensitivity : 0.9479          
            Specificity : 0.9272          
         Pos Pred Value : 0.8830          
         Neg Pred Value : 0.9684          
             Prevalence : 0.3670          
         Detection Rate : 0.3478          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.9375          
                                          
       'Positive' Class : spam            
               

In [174]:
qda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     558  139
   spam         25  428
                                          
               Accuracy : 0.8574          
                 95% CI : (0.8358, 0.8771)
    No Information Rate : 0.507           
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7139          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.7549          
            Specificity : 0.9571          
         Pos Pred Value : 0.9448          
         Neg Pred Value : 0.8006          
             Prevalence : 0.4930          
         Detection Rate : 0.3722          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.8560          
                                          
       'Positive' Class : spam            
               

### Only freqs log

In [183]:
spam_freq <- spam_log[, -seq(53, 55)]

In [184]:
train_freq <- spam_freq[train_indices, ]
test_freq  <- spam_freq[-train_indices, ]

In [185]:
train_freq <- data.frame(train_freq)
test_freq <- data.frame(test_freq)
lda_fit <- lda(type ~ ., data = train_freq)
lda_predictions <- predict(lda_fit, test_freq)

In [186]:
qda_fit <- qda(type ~ ., data = train_freq)
qda_predictions <- predict(qda_fit, test_freq)

In [187]:
lda_cm <- confusionMatrix(test_freq$type, lda_predictions$class, positive = "spam")

In [188]:
qda_cm <- confusionMatrix(test_freq$type, qda_predictions$class, positive = "spam")

In [189]:
lda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     670   27
   spam         56  397
                                          
               Accuracy : 0.9278          
                 95% CI : (0.9113, 0.9421)
    No Information Rate : 0.6313          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.8471          
                                          
 Mcnemar's Test P-Value : 0.002116        
                                          
            Sensitivity : 0.9363          
            Specificity : 0.9229          
         Pos Pred Value : 0.8764          
         Neg Pred Value : 0.9613          
             Prevalence : 0.3687          
         Detection Rate : 0.3452          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.9296          
                                          
       'Positive' Class : spam            
               

In [190]:
qda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     552  145
   spam         24  429
                                         
               Accuracy : 0.853          
                 95% CI : (0.8312, 0.873)
    No Information Rate : 0.5009         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.706          
                                         
 Mcnemar's Test P-Value : < 2.2e-16      
                                         
            Sensitivity : 0.7474         
            Specificity : 0.9583         
         Pos Pred Value : 0.9470         
         Neg Pred Value : 0.7920         
             Prevalence : 0.4991         
         Detection Rate : 0.3730         
   Detection Prevalence : 0.3939         
      Balanced Accuracy : 0.8529         
                                         
       'Positive' Class : spam           
                                   

### Only capital

In [191]:
spam_capital <- spam_log[, seq(53, 56)]

In [192]:
train_capital <- spam_capital[train_indices, ]
test_capital  <- spam_capital[-train_indices, ]

In [193]:
train_capital <- data.frame(train_capital)
test_capital <- data.frame(test_capital)
lda_fit <- lda(type ~ ., data = train_capital)
lda_predictions <- predict(lda_fit, test_capital)

In [194]:
qda_fit <- qda(type ~ ., data = train_capital)
qda_predictions <- predict(qda_fit, test_capital)

In [198]:
lda_cm <- confusionMatrix(test_capital$type, lda_predictions$class, positive = "spam")
qda_cm <- confusionMatrix(test_capital$type, qda_predictions$class, positive = "spam")

In [199]:
lda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     595  102
   spam        208  245
                                          
               Accuracy : 0.7304          
                 95% CI : (0.7038, 0.7559)
    No Information Rate : 0.6983          
    P-Value [Acc > NIR] : 0.009007        
                                          
                  Kappa : 0.4113          
                                          
 Mcnemar's Test P-Value : 2.467e-09       
                                          
            Sensitivity : 0.7061          
            Specificity : 0.7410          
         Pos Pred Value : 0.5408          
         Neg Pred Value : 0.8537          
             Prevalence : 0.3017          
         Detection Rate : 0.2130          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.7235          
                                          
       'Positive' Class : spam            
               

In [200]:
qda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     619   78
   spam        247  206
                                          
               Accuracy : 0.7174          
                 95% CI : (0.6904, 0.7433)
    No Information Rate : 0.753           
    P-Value [Acc > NIR] : 0.9974          
                                          
                  Kappa : 0.3668          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.7254          
            Specificity : 0.7148          
         Pos Pred Value : 0.4547          
         Neg Pred Value : 0.8881          
             Prevalence : 0.2470          
         Detection Rate : 0.1791          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.7201          
                                          
       'Positive' Class : spam            
               

### Features highly correlated with spam

In [201]:
spam_cor_feat <- spam_log[, c("your", "num000", "remove", "charDollar", "charExclamation", "type")]

In [202]:
train_cor <- data.frame(spam_cor_feat[train_indices, ])
test_cor <- data.frame(spam_cor_feat[-train_indices, ])

In [203]:
lda_fit <- lda(type ~ ., data = train_cor)
lda_predictions <- predict(lda_fit, test_cor)
qda_fit <- qda(type ~ ., data = train_cor)
qda_predictions <- predict(qda_fit, test_cor)

In [204]:
lda_cm <- confusionMatrix(test_cor$type, lda_predictions$class, positive = "spam")
qda_cm <- confusionMatrix(test_cor$type, qda_predictions$class, positive = "spam")

In [205]:
lda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     666   31
   spam         91  362
                                          
               Accuracy : 0.8939          
                 95% CI : (0.8747, 0.9111)
    No Information Rate : 0.6583          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7726          
                                          
 Mcnemar's Test P-Value : 9.213e-08       
                                          
            Sensitivity : 0.9211          
            Specificity : 0.8798          
         Pos Pred Value : 0.7991          
         Neg Pred Value : 0.9555          
             Prevalence : 0.3417          
         Detection Rate : 0.3148          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.9005          
                                          
       'Positive' Class : spam            
               

In [206]:
qda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     650   47
   spam         91  362
                                          
               Accuracy : 0.88            
                 95% CI : (0.8598, 0.8982)
    No Information Rate : 0.6443          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7443          
                                          
 Mcnemar's Test P-Value : 0.0002518       
                                          
            Sensitivity : 0.8851          
            Specificity : 0.8772          
         Pos Pred Value : 0.7991          
         Neg Pred Value : 0.9326          
             Prevalence : 0.3557          
         Detection Rate : 0.3148          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.8811          
                                          
       'Positive' Class : spam            
               

### Highest difference in average frequency

In [207]:
spam_diff_feat <- spam_log[, c("george", "you", "your", "hp", "free", "hpl", "charExclamation", "our", "re", "edu", "type")]

In [208]:
train_diff <- data.frame(spam_diff_feat[train_indices, ])
test_diff <- data.frame(spam_diff_feat[-train_indices, ])

In [209]:
lda_fit <- lda(type ~ ., data = train_diff)
lda_predictions <- predict(lda_fit, test_diff)
qda_fit <- qda(type ~ ., data = train_diff)
qda_predictions <- predict(qda_fit, test_diff)

In [210]:
lda_cm <- confusionMatrix(test_diff$type, lda_predictions$class, positive = "spam")
qda_cm <- confusionMatrix(test_diff$type, qda_predictions$class, positive = "spam")

In [211]:
lda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     644   53
   spam         75  378
                                          
               Accuracy : 0.8887          
                 95% CI : (0.8691, 0.9063)
    No Information Rate : 0.6252          
    P-Value [Acc > NIR] : < 2e-16         
                                          
                  Kappa : 0.7649          
                                          
 Mcnemar's Test P-Value : 0.06343         
                                          
            Sensitivity : 0.8770          
            Specificity : 0.8957          
         Pos Pred Value : 0.8344          
         Neg Pred Value : 0.9240          
             Prevalence : 0.3748          
         Detection Rate : 0.3287          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.8864          
                                          
       'Positive' Class : spam            
               

In [212]:
qda_cm

Confusion Matrix and Statistics

          Reference
Prediction nonspam spam
   nonspam     478  219
   spam         19  434
                                          
               Accuracy : 0.793           
                 95% CI : (0.7685, 0.8161)
    No Information Rate : 0.5678          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.5977          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.6646          
            Specificity : 0.9618          
         Pos Pred Value : 0.9581          
         Neg Pred Value : 0.6858          
             Prevalence : 0.5678          
         Detection Rate : 0.3774          
   Detection Prevalence : 0.3939          
      Balanced Accuracy : 0.8132          
                                          
       'Positive' Class : spam            
               